# Dinâmica simbólica em Python

Este notebook demonstra como calcular as matrizes **M**, **C**, **H** e **G** para um robô de cadeia aberta usando os módulos Python criados a partir do projeto em MATLAB. A estrutura separa o código em vários arquivos `.py` e o notebook atua apenas como orquestrador.

### 1. Clonar o repositório

Você pode clonar seu repositório usando o comando `!git clone` no Colab.

In [ ]:
# Substitua esta URL pela URL do seu repositório Git
repo_url = "https://github.com/danisilva1110-spec/PFPython.git"

# Nome da pasta de destino (derivado automaticamente da URL)
repo_dir = repo_url.rstrip("/").split("/")[-1]
if repo_dir.endswith(".git"):
    repo_dir = repo_dir[:-4]

import sys
from pathlib import Path

current_repo = Path.cwd()
local_package = current_repo / "robot_dynamics"

# Se o notebook já estiver dentro do repositório, use-o diretamente
if local_package.exists():
    repo_path = current_repo
else:
    repo_path = current_repo / repo_dir
    if not repo_path.exists():
        # Clone o repositório apenas se ele ainda não existir
        !git clone {repo_url} {repo_dir}

# Garanta que o repositório esteja no sys.path para que os módulos sejam encontrados
sys.path.insert(0, str(repo_path.resolve()))

Cloning into 'PFPython'...
remote: Enumerating objects: 349, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 349 (delta 21), reused 2 (delta 2), pack-reused 296 (from 1)
Receiving objects: 100% (349/349), 437.00 KiB | 3.44 MiB/s, done.
Resolving deltas: 100% (186/186), done.


In [ ]:
from sympy import simplify, symbols

from robot_dynamics import (
    forward_kinematics,
    inertia_matrix,
    coriolis_matrix,
    centripetal_vector,
    gravity_vector,
    kinetic_energy,
    planar_two_dof,
)

### 2. Listar o conteúdo do repositório clonado
Após clonar (ou ao usar este notebook diretamente dentro do repositório), a variável `repo_path` aponta para a pasta onde o código está. Você pode inspecionar os arquivos com o comando abaixo.


In [ ]:
# Substitua 'repo_path' pelo caminho do repositório se você tiver alterado as variáveis acima
!ls {repo_path}

main.ipynb  README.md  robot_dynamics


## Variáveis automáticas e máscara 0/1

`equations_of_motion_from_order` gera `q`/`qd` a partir da ordem de juntas (Dx/Dy/Dz/x/y/z) e aceita um `active_mask` para desligar DOF sem mexer nas tabelas. O exemplo abaixo segue a ordem do UVMS do MATLAB e usa máscara para travar as três translações do ROV.


In [ ]:
from sympy import Matrix, symbols
from robot_dynamics import equations_of_motion_from_order, mask_excentricities_by_axis

axis_order = ["Dx", "Dy", "Dz", "z", "y", "x", "z", "y", "y", "z", "y", "z"]
active_mask = [0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1]

# DH placeholders (alinhados com axis_order)
dh_params = [(0, 0, 0, 0) for _ in axis_order]

# Matriz genérica de comprimentos (Lx_i, Ly_i, Lz_i)
Lx = symbols("Lx1:13")
Ly = symbols("Ly1:13")
Lz = symbols("Lz1:13")
matriz_comprimentos = list(zip(Lx, Ly, Lz))

# Zera automaticamente os comprimentos fora do eixo de cada junta
matriz_excentricidades = mask_excentricities_by_axis(matriz_comprimentos, axis_order)

# Tensores de inércia identidade (placeholders)
I3 = Matrix.eye(3)
inercia = [I3] * len(axis_order)

m = symbols("m1:13")
g = symbols("g")
gravity = Matrix([0, 0, -g])

resultado_mask = equations_of_motion_from_order(
    dh_params,
    axis_order,
    m,
    matriz_excentricidades,
    inercia,
    gravity=gravity,
    active_mask=active_mask,
)

matriz_excentricidades


Matrix([
[0.5*m10*(18*sin(x + 3*y + 3*z)**2 + 18*cos(x + 3*y + 3*z)**2) + 0.5*m8*(8*sin(x + 2*y + 2*z)**2 + 8*cos(x + 2*y + 2*z)**2) + 0.5*m9*(8*sin(x + 3*y + 2*z)**2 + 8*cos(x + 3*y + 2*z)**2) + 49.0,                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

NameError: name 'resultado_mask' is not defined